
# Random Acts of Pizza

1. [Introduction](#Introduction)
2. [Exploratory Data Analysis](#Exploratory-Data-Analysis)
3. [Text Classification](#Text-Classification)
4. [Non-Text Classification](#Non-Text-Classification)<br>
    4.1 [Feature Selection](#Feature-Selection)<br>
    4.2 [Logistic Regression Model](#Logistic-Regression-Model)
5. [Building an Ensemble Classifier](#Building-an-Ensemble-Classifier)
6. [Kaggle Submission](#Kaggle-Submission)
7. [Conclusion](#Conclusion)

## Introduction

Reddit's Random Acts of Pizza subreddit (https://www.reddit.com/r/Random_Acts_Of_Pizza/) is where people can request free pizza from others on the Internet. The objective of this exercise is to predict which pizza requests will be successful based on the features of those requests. 

This dataset is available in Kaggle - https://www.kaggle.com/c/random-acts-of-pizza

In [248]:
import scipy
import datetime
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import json
import random
import re
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from scipy.optimize import minimize
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import itertools
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import textstat
import warnings
warnings.filterwarnings('ignore')

train_df = pd.read_json("data/train.json")
test_df = pd.read_json("data/test.json")

print ("Training Data shape : %s" % str(train_df.shape))
print ("Test Data shape : %s" % str(test_df.shape))

Training Data shape : (4040, 32)
Test Data shape : (1631, 17)


Our Training dataset has 4040 records and 32 features and our Test dataset has 1631 records and 17 features. The 17 features in the Test dataset are what we will need to work on to make a Kaggle submission eventually. We will get rid of the extra features in the Training data set later in this notebook.

We define some utlity functions next that we will use later in this notebook - 'expresses_locality' that detects if a text contains US locality information like state, city etc. and 'evaluate' that helps evaluates a model.

In [249]:
#The 2 class names for this exercise.
class_names = ['No-Pizza','Pizza']
#Utility method that we are goind to evaluate our models.
def evaluate(classes,y_true,y_pred,
                          normalize=True,
                          title='Confusion matrix'):
    print (classification_report(y_true, y_pred))
    
    cm=confusion_matrix(y_true,y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cmap=plt.cm.Blues
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()
    print ("Metrics.roc_auc_score : %f" % metrics.roc_auc_score(y_true, y_pred))

with open('states.json') as f:
    states = json.load(f)
with open('countries.json') as f:
    countries = json.load(f)

# check whether string contains state or country
def expresses_locality(string):
    for state in states.values():
        if state.lower() in string.lower():
            return True
    for country in countries.values():
        if country.lower() in string.lower():
            return True
    for state_abbrev in states.keys():
        abbrevs = []
        abbrevs.append(", " + state_abbrev + " ")
        abbrevs.append("," + state_abbrev + " ")
        for abbrev in abbrevs:
            if abbrev in string:
                return True
    return False

## Exploratory Data Analysis

In [250]:
train_df.groupby(['requester_received_pizza']).size()

requester_received_pizza
False    3046
True      994
dtype: int64

We observe that around 1 in 4 requests are successful so our classes are skewed with 994 success records and 3046 failed records in our Training dataset. Let us balance the classes by oversampling the success records.

In [251]:
train_df_fail = train_df[train_df['requester_received_pizza'] == 0]
train_df_success = train_df[train_df['requester_received_pizza'] == 1]
train_df = pd.concat([train_df_fail, train_df_success,train_df_success,train_df_success,train_df_success.sample(len(train_df_fail.index)-3*len(train_df_success.index))], axis = 0)
train_df.groupby(['requester_received_pizza']).size()

requester_received_pizza
False    3046
True     3046
dtype: int64

Our classes are balanced now with 3046 records each. Let us view the histograms of the columns in our training dataset.

In [252]:
#train_df.hist(figsize=(30,30))
#plt.show()

Let us view the correlation information for the features in our training dataset:

In [253]:
corr_matrix = train_df.corr()
corr_matrix['requester_received_pizza'].sort_values(ascending=False)

requester_received_pizza                                1.000000
requester_number_of_posts_on_raop_at_retrieval          0.428021
request_number_of_comments_at_retrieval                 0.304436
requester_number_of_comments_in_raop_at_retrieval       0.245899
requester_days_since_first_post_on_raop_at_retrieval    0.150272
requester_account_age_in_days_at_retrieval              0.135530
requester_number_of_posts_on_raop_at_request            0.127648
requester_number_of_comments_at_retrieval               0.120138
requester_number_of_comments_in_raop_at_request         0.117235
number_of_upvotes_of_request_at_retrieval               0.109903
requester_days_since_first_post_on_raop_at_request      0.098886
requester_upvotes_minus_downvotes_at_retrieval          0.077707
post_was_edited                                         0.056427
requester_number_of_posts_at_retrieval                  0.054621
requester_account_age_in_days_at_request                0.046126
requester_number_of_subre

## Text Classification

In [254]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

def text_preprocessor(s):
    message = s.lower()
    message = re.sub(r"\[|\]|\(|\)|\$|\!|\/|\.|\*|\+|\&|\=|\%|\:|\?|\"|\,|\;|\@|\_|\\|\}|\{|\||\~|\'", " ", message)
    message = re.sub(r"[0-9]", " ", message)
    message = re.sub(r"[-]*", "", message)
    message = ' '.join([word[0:20] for word in message.split() if len(word)>3])
    return message

def stemming_tokenizer(str_input):
    words = str_input.split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [255]:
from sklearn.feature_extraction.text import *

data = pd.read_json("data/train.json")
#all_data = data[data["requester_received_pizza"]==True].iloc[0:2000]
#all_data = all_data.append(data[data["requester_received_pizza"]==False].iloc[0:2000])
#data = all_data.sample(frac=1)

train_data, dev_data = train_test_split(data, test_size=0.3)
train_labels = train_data["requester_received_pizza"]
dev_labels = dev_data["requester_received_pizza"]
train_data = train_data['request_text_edit_aware']
dev_data = dev_data['request_text_edit_aware']

cv = TfidfVectorizer(stop_words='english',
                    preprocessor=text_preprocessor,
                    lowercase=True,
                    tokenizer=stemming_tokenizer,
                    min_df=5, 
                    max_df=0.2, 
                    ngram_range=(1,3),
                    strip_accents='ascii',
                    use_idf=False
                    )
transformer = cv.fit_transform(train_data)
clf = LogisticRegression()
clf.fit(transformer,train_labels)
dev_data_trans = cv.transform(dev_data)
y_pred = clf.predict(dev_data_trans)

print ("Accuracy (on dev set): %.4f" % metrics.accuracy_score(y_true=dev_labels, y_pred=y_pred))
print (metrics.classification_report(y_true=dev_labels, y_pred=y_pred))

transformer = cv.transform(train_data)
print transformer.shape

Accuracy (on dev set): 0.7409
              precision    recall  f1-score   support

       False       0.75      0.99      0.85       904
        True       0.31      0.02      0.03       308

   micro avg       0.74      0.74      0.74      1212
   macro avg       0.53      0.50      0.44      1212
weighted avg       0.64      0.74      0.64      1212

(2828, 3372)


In [256]:
print len(cv.get_feature_names())

3372


## Non-Text Classification

Let us extract features from the non-textual data that could be useful to build our model.

### Feature Selection

In [257]:
def create_features(df):
    
    # Whether or not the user was subscribed to exactly 0 subreddits
    df['zero_subreddits'] = df['requester_number_of_subreddits_at_request'] == 0

    # Total number of comments + number of random acts of pizza comments
    df['total_comments'] = (df['requester_number_of_comments_in_raop_at_request'] +
                                 df['requester_number_of_comments_at_request'])

    # The ratio of the user's total comments to their number of random acts of pizza comments
    df['comment_ratio'] = (df['total_comments'] /
                                df['requester_number_of_comments_in_raop_at_request'])
    
    df.loc[df['comment_ratio'] == np.inf, 'comment_ratio'] = \
        df.loc[df['comment_ratio'] != np.inf, 'comment_ratio'].mean(skipna = True)
    
    df.loc[pd.isnull(df['comment_ratio']), 'comment_ratio'] = \
        df.loc[pd.notnull(df['comment_ratio']), 'comment_ratio'].mean(skipna = True)

    # The number of upvotes they've received
    df['upvotes'] = (df['requester_upvotes_minus_downvotes_at_request'] +
                       df['requester_upvotes_plus_downvotes_at_request']) / 2

    # The number of downvotes they've received
    df['downvotes'] = (df['requester_upvotes_plus_downvotes_at_request'] -
                             df['upvotes'])
    
    # The ratio of upvotes they've received
    df['upvote_ratio'] = (df['upvotes'] /
                               (df['upvotes'] + df['downvotes']))
    
    df.loc[df['upvote_ratio'] == np.inf, 'upvote_ratio'] = \
        df.loc[df['upvote_ratio'] != np.inf, 'upvote_ratio'].mean(skipna = True)

    df.loc[pd.isnull(df['upvote_ratio']), 'upvote_ratio'] = \
        df.loc[pd.notnull(df['upvote_ratio']), 'upvote_ratio'].mean(skipna = True)

    # Get the date in order to make future variables
    df['date'] = pd.to_datetime(df['unix_timestamp_of_request_utc'], unit = 's')

    # Hour of the post
    df['hour'] = pd.DatetimeIndex(df['date']).hour

    # Day of the post
    df['day'] = pd.DatetimeIndex(df['date']).day

    # The post's day of the week
    df['weekday'] = pd.DatetimeIndex(df['date']).weekday

    # Whether the post was made in the first half of the month
    df['first_half_of_month'] = df['day'] <= 15
    df['first_half_of_month'] = df['first_half_of_month'].astype(int)

    # Whether the post was made on a weekend
    df['weekend'] = (df['weekday'] == 5) | (df['weekday'] == 6)
    df['weekend'] = df['weekend'].astype(int)

    # Whether the post was made in the morning
    df['morning'] = (df['hour'] >= 6) & (df['hour'] < 12)
    df['morning'] = df['morning'].astype(int)

    # Whether the post was made in the afternoon
    df['afternoon'] = (df['hour'] >= 12) & (df['hour'] < 16)
    df['afternoon'] = df['afternoon'].astype(int)

    # Whether the post was made in the evening
    df['evening'] = (df['hour'] >= 16) & (df['hour'] < 20)
    df['evening'] = df['evening'].astype(int)

    # Whether the post was made at night
    df['night'] = (df['hour'] >= 20) & (df['hour'] < 23)
    df['night'] = df['night'].astype(int)

    # Whether the post was made late at night
    df['latenight'] = (df['hour'] >= 23) & (df['hour'] < 6)
    df['latenight'] = df['latenight'].astype(int)

    # Whether there was a difference between the utc and unix timestamp
    df['utcdiff'] = (df['unix_timestamp_of_request_utc'] -
                              df['unix_timestamp_of_request'])

    # Month of the post
    df['month'] = pd.DatetimeIndex(df['date']).month

    # Week of the post
    df['week'] = pd.DatetimeIndex(df['date']).week
    
    # Get the US federal holidays and categorize the request dates as holiday or not.
    cal = USFederalHolidayCalendar()
    holiday_list = cal.holidays(start=df['date'].min(),
                            end=df['date'].max())
    holiday_list = [time.date() for time in holiday_list]

    df['justdate'] = [time.date() for time in df['date']]
    df['holiday'] = df['justdate'].isin(holiday_list)
    df['holiday'] = df['holiday'].astype(int)

    return(df)

train_df = create_features(train_df)
test_df = create_features(test_df)

Then we extract features from the textual data, mainly the title and the request text fields. We use the NTLK package to compute sentiment scores on the request text and the textstat package to compute further extract attributes from the text of the title and the request text.


In [258]:
### Extract the user request text so that we can create word features out
### of it for inclusion in the training since we can't use whole text.
train_request_text = train_df[['request_text_edit_aware']]
test_request_text = test_df[['request_text_edit_aware']]

### Code to add in text features
def create_text_features(df):
    # Training data with engineered features
    df_new = df
    
    # Sentiment analysis for the posts. We are looking at only posts that have a high postive or negative sentiment
    # and appending them as features columns 'positive_sentiment' and 'negative_sentiment' respectively.
    sid = SentimentIntensityAnalyzer()
    df_new['positive_sentiment']=df.request_text_edit_aware.apply(lambda x: sid.polarity_scores(x)['compound']>=0.5)
    df_new['negative_sentiment']=df.request_text_edit_aware.apply(lambda x: sid.polarity_scores(x)['compound']<=-0.5)
    
    # Length of the post's title
    df_new['title_len'] = df.request_title.str.len()

    # Length of the post's text
    df_new['request_len'] = df.request_text_edit_aware.str.len()

    # Number of words in title
    df_new['title_words'] = df['request_title'].apply(lambda x: len(x.split(' ')))

    # Number of words in body
    df_new['body_words'] = df['request_text_edit_aware'].apply(lambda x: len(x.split(' ')))
    
    # Number of sentences in body
    sen_count = re.compile(r'([a-zA-Z][^\.!?]*[\.!?])', re.M)
    df_new['body_sentences'] = df['request_text_edit_aware'].apply(lambda x: len(sen_count.findall(x)))

    # Post includes an image
    image_count = re.compile(r'(imgur\.com|\.jpg)', re.IGNORECASE)
    images = df['request_text_edit_aware'].apply(lambda x: len(image_count.findall(x)))
    df_new['has_image'] = np.where(images > 0,1,0)

    # Post includes a 'tl;dr'
    tldr_count = re.compile(r'(tl;dr|tldr|tl,dr|tl:dr)', re.IGNORECASE)
    tldrs = df['request_text_edit_aware'].apply(lambda x: len(tldr_count.findall(x)))
    df_new['has_tldr'] = np.where(tldrs > 0,1,0)

    # More than zero all-caps words in title (min char = 3)
    df_new['title_caps'] = df['request_title'].apply( \
        lambda x: np.where(sum(1 for c in re.split('\W+',x) if c.isupper() and len(c) > 3) > 0, 1,0))

    # More than zero all-caps words in body (min char = 2)
    df_new['body_caps'] = df['request_text_edit_aware'].apply( \
        lambda x: np.where(sum(1 for c in re.split('\W+',x) if c.isupper() and len(c) > 2) > 0, 1,0))

    # Exclamation marks in title
    excl_marks = re.compile(r'(!)', re.M)
    df_new['excl_marks_title'] = df['request_title'].apply(lambda x: len(excl_marks.findall(x)))
    # print train_df_f['excl_marks']

    # Exclamation marks in body
    df_new['excl_marks_body'] = df['request_text_edit_aware'].apply(lambda x: len(excl_marks.findall(x)))

    # Dollar signs in title
    dollar_signs = re.compile(r'(\$|dollar)', re.IGNORECASE)
    df_new['dollars_title'] = df['request_title'].apply(lambda x: len(dollar_signs.findall(x)))

    # Dollar signs in body
    df_new['dollars_body'] = df['request_text_edit_aware'].apply(lambda x: len(dollar_signs.findall(x)))
    
    # Categories
    desire = re.compile(r'(christmas|hungry|xmas|ramen|friend|party|birthday|boyfriend|girlfriend|date|drinks|drunk|wasted|invite|invited|celebrate|celebrating|game|games|movie|beer|crave|craving|starving)', re.IGNORECASE)
    family = re.compile(r'(husband|wife|family|parent|parents|mother|father|mom|mum|son|dad|daughter)', re.IGNORECASE)
    job = re.compile(r'(job|unemployment|employment|hire|hired|fired|interview|work|paycheck)', re.IGNORECASE)
    money = re.compile(r'(foodstamps|money|bill|bills|rent|bank|account|paycheck|due|broke|bills|deposit|cash|dollar|dollars|bucks|paid|payed|buy|check|spent|financial|poor|loan|credit|budget|day|now| \
        time|week|until|last|month|tonight|today|next|night|when|tomorrow|first|after|while|before|long|hour|Friday|ago|still|due|past|soon|current|years|never|till|yesterday|morning|evening)', re.IGNORECASE)
    student = re.compile(r'(college|student|university|finals|study|studying|class|semester|school|roommate|project|tuition|dorm)', re.IGNORECASE)
    health = re.compile(r'(under the weather|sick|fever|flu|cough|cancer|cold)', re.IGNORECASE)
    
    df_new['desire_category'] = df['request_text_edit_aware'].apply(lambda x: len(desire.findall(x)))
    df_new['family_category'] = df['request_text_edit_aware'].apply(lambda x: len(family.findall(x)))
    df_new['job_category'] = df['request_text_edit_aware'].apply(lambda x: len(job.findall(x)))
    df_new['money_category'] = df['request_text_edit_aware'].apply(lambda x: len(money.findall(x)))
    df_new['student_category'] = df['request_text_edit_aware'].apply(lambda x: len(student.findall(x)))
    df_new['health'] = df['request_text_edit_aware'].apply(lambda x: len(health.findall(x)))
    
    # Gratitude
    gratitude = re.compile(r'(thank|thanks|thankful|appreciate|grateful|gratitude|advance)',re.IGNORECASE)
    df_new['gratitude'] = df['request_text_edit_aware'].apply(lambda x: np.where(len(gratitude.findall(x)) > 0, 1, 0))
    
    # Reciprocity
    reciprocity = re.compile(r'(pay it forward|pay forward|paid it forward|pay the act forward|pay the favor back|paying it forward|pay this forward|pay pizza forward|pay back|pay it back|pay you back|return the favor|return the favour|pay a pizza forward|repay|reciprocate)',re.IGNORECASE)    
    df_new['reciprocity'] = df['request_text_edit_aware'].apply(lambda x: np.where(len(reciprocity.findall(x))> 0, 1, 0))
    
    df_new['locality_in_title']=df.apply(lambda x: 1 if expresses_locality(x['request_title']) else 0, axis=1).astype(int)
    df_new['locality_in_text']=df.apply(lambda x: 1 if expresses_locality(x['request_text_edit_aware']) else 0, axis=1).astype(int)
    df_new['flesch_reading_ease_text']=df.apply(lambda x: textstat.flesch_reading_ease(x['request_text_edit_aware']), axis=1).astype(float)
    df_new['flesch_reading_ease_title']=df.apply(lambda x: textstat.flesch_reading_ease(x['request_title']), axis=1).astype(float)
    df_new['smog_index_text']=df.apply(lambda x: textstat.smog_index(x['request_text_edit_aware']), axis=1).astype(float)
    df_new['smog_index_title']=df.apply(lambda x: textstat.smog_index(x['request_title']), axis=1).astype(float)
    df_new['flesch_kincaid_grade_text']=df.apply(lambda x: textstat.flesch_kincaid_grade(x['request_text_edit_aware']), axis=1).astype(float)
    df_new['flesch_kincaid_grade_title']=df.apply(lambda x: textstat.flesch_kincaid_grade(x['request_title']), axis=1).astype(float)
    df_new['coleman_liau_index_text']=df.apply(lambda x: textstat.coleman_liau_index(x['request_text_edit_aware']), axis=1).astype(float)
    df_new['coleman_liau_index_title']=df.apply(lambda x: textstat.coleman_liau_index(x['request_title']), axis=1).astype(float)
    df_new['automated_readability_index_text']=df.apply(lambda x: textstat.automated_readability_index(x['request_text_edit_aware']), axis=1).astype(float)
    df_new['automated_readability_index_title']=df.apply(lambda x: textstat.automated_readability_index(x['request_title']), axis=1).astype(float)
    df_new['dale_chall_readability_score_text']=df.apply(lambda x: textstat.dale_chall_readability_score(x['request_text_edit_aware']), axis=1).astype(float)
    df_new['dale_chall_readability_score_title']=df.apply(lambda x: textstat.dale_chall_readability_score(x['request_title']), axis=1).astype(float)
    df_new['difficult_words_text']=df.apply(lambda x: textstat.difficult_words(x['request_text_edit_aware']), axis=1).astype(int)
    df_new['difficult_words_title']=df.apply(lambda x: textstat.difficult_words(x['request_title']), axis=1).astype(int)
    df_new['linsear_write_formula_text']=df.apply(lambda x: textstat.linsear_write_formula(x['request_text_edit_aware']), axis=1).astype(float)
    df_new['linsear_write_formula_title']=df.apply(lambda x: textstat.linsear_write_formula(x['request_title']), axis=1).astype(float)
    df_new['gunning_fog_text']=df.apply(lambda x: textstat.gunning_fog(x['request_text_edit_aware']), axis=1).astype(float)
    df_new['gunning_fog_title']=df.apply(lambda x: textstat.gunning_fog(x['request_title']), axis=1).astype(float)
    df_new['text_standard_text']=df.apply(lambda x: textstat.text_standard(x['request_text_edit_aware'],float_output=True), axis=1).astype(float)
    df_new['text_standard_title']=df.apply(lambda x: textstat.text_standard(x['request_title'],float_output=True), axis=1).astype(float)
    df_new['sentence_count_text']=df.apply(lambda x: textstat.sentence_count(x['request_text_edit_aware']), axis=1).astype(int)
    df_new['sentence_count_title']=df.apply(lambda x: textstat.sentence_count(x['request_title']), axis=1).astype(int)
    
    return(df_new)

train_df = create_text_features(train_df)
test_df = create_text_features(test_df)


In [260]:
def create_tfidf_features(df):
    
    train_request_text = train_df['request_text_edit_aware']
    cv = TfidfVectorizer(stop_words='english',
                    preprocessor=text_preprocessor,
                    lowercase=True,
                    tokenizer=stemming_tokenizer,
                    min_df=10, 
                    max_df=0.1, 
                    ngram_range=(1,3),
                    strip_accents='ascii',
                    use_idf=False
                    )
    transformer = cv.fit_transform(train_request_text)
    sdf = pd.SparseDataFrame(transformer,default_fill_value=0)
    print sdf.shape
    return sdf

sdf = create_tfidf_features(train_request_text)


(6092, 3682)


In [261]:
sdf[0]

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
5       0.000000
6       0.000000
7       0.000000
8       0.000000
9       0.000000
10      0.000000
11      0.000000
12      0.000000
13      0.000000
14      0.000000
15      0.000000
16      0.000000
17      0.000000
18      0.000000
19      0.000000
20      0.000000
21      0.000000
22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.000000
30      0.000000
31      0.000000
32      0.000000
33      0.000000
34      0.000000
35      0.000000
36      0.000000
37      0.000000
38      0.000000
39      0.000000
40      0.000000
41      0.000000
42      0.000000
43      0.000000
44      0.000000
45      0.000000
46      0.000000
47      0.000000
48      0.000000
49      0.000000
50      0.000000
51      0.000000
52      0.000000
53      0.000000
54      0.000000
55      0.000000
56      0.000000
57      0.000000
58      0.0000

In [262]:
sdf = sdf.loc[~sdf.index.duplicated(keep='first')]
train_df = train_df.loc[~train_df.index.duplicated(keep='first')]
train_df = pd.concat([train_df, sdf], axis=1)

In [269]:
pos_emoji = re.compile(r'(:\))', re.IGNORECASE)
train_df['pos_emoji'] = train_df['request_text_edit_aware'].apply(lambda x: len(pos_emoji.findall(x)))
test_df['pos_emoji'] = test_df['request_text_edit_aware'].apply(lambda x: len(pos_emoji.findall(x)))

TypeError: expected string or buffer

We will convert the categorical variables ['hour', 'week', 'day', 'weekday', 'month', 'utcdiff'] into binary variables.

In [264]:
train_data_categorical = train_df[['hour', 'week', 'day', 'weekday', 'month', 'utcdiff']]
test_data_categorical = test_df[['hour', 'week', 'day', 'weekday', 'month', 'utcdiff']]

train_data = train_df.drop(['request_text', 
                            'giver_username_if_known', 'post_was_edited',
                            'request_id', 'request_text_edit_aware',
                            'request_title',
                            'requester_subreddits_at_request',
                            'unix_timestamp_of_request_utc',
                            'unix_timestamp_of_request',
                            'requester_username', 'requester_user_flair',
                            'number_of_downvotes_of_request_at_retrieval',
                            'number_of_upvotes_of_request_at_retrieval',
                            'request_number_of_comments_at_retrieval',
                            'requester_account_age_in_days_at_retrieval',
                            'requester_days_since_first_post_on_raop_at_retrieval',
                            'requester_number_of_comments_at_retrieval',
                            'requester_number_of_comments_in_raop_at_retrieval',
                            'requester_number_of_posts_at_retrieval',
                            'requester_number_of_posts_on_raop_at_retrieval',
                            'requester_upvotes_minus_downvotes_at_retrieval',
                            'requester_upvotes_plus_downvotes_at_retrieval',
                            'date', 'hour', 'week', 'day', 'weekday', 'month', 'utcdiff', 'justdate'
                            ],
                           axis = 1)

test_data = test_df.drop(['giver_username_if_known', 'request_id',
                          'request_text_edit_aware',
                            'request_title',
                            'requester_subreddits_at_request',
                            'unix_timestamp_of_request_utc',
                            'unix_timestamp_of_request',
                            'requester_username',
                            'date', 'hour', 'week', 'day', 'weekday', 'month', 'utcdiff', 'justdate'
                            ],
                           axis = 1)

train_data_dummy = pd.DataFrame(index = train_data.index)
test_data_dummy = pd.DataFrame(index = test_data.index)

for column in train_data_categorical.columns:
    train_data_dummy = pd.concat([train_data_dummy,
                                  pd.get_dummies(train_data_categorical[column],
                                                prefix = column)], axis = 1)    
    test_data_dummy = pd.concat([test_data_dummy,
                                 pd.get_dummies(test_data_categorical[column],
                                                prefix = column)], axis = 1)

train_data = pd.concat([train_data, train_data_dummy], axis = 1)
test_data = pd.concat([test_data, test_data_dummy], axis = 1)

Let us do a 70:30 split to create the training and validation data sets, and extract the 'requester_received_pizza' column to create the train_labels and dev_labels dataframes.

In [ ]:
sum(train_data["neg_emoji"])

In [265]:
train_data1, dev_data1 = train_test_split(train_data, test_size=0.30)
train_labels = np.ravel(train_data1[['requester_received_pizza']])
train_data1 = train_data1.drop(['requester_received_pizza'],axis = 1)
dev_labels = np.ravel(dev_data1[['requester_received_pizza']])
dev_data1 = dev_data1.drop(['requester_received_pizza'],axis = 1)

print ("Training Data shape : %s" % str(train_data1.shape))
print ("Validation Data shape : %s" % str(dev_data1.shape))

Training Data shape : (4264, 3881)
Validation Data shape : (1828, 3881)


In [266]:
train_data1[0]

2115    0.000000
1692    0.000000
4565    0.000000
331     0.000000
4817    0.000000
4116    0.000000
845     0.000000
702     0.000000
786     0.000000
1420    0.000000
2344    0.000000
144     0.000000
1931    0.000000
1481    0.000000
3196    0.000000
5036    0.000000
5935    0.000000
2551    0.000000
2998    0.000000
5138    0.000000
5348    0.000000
1388    0.000000
4218    0.000000
3870    0.000000
3819    0.000000
1849    0.000000
5259    0.000000
4704    0.000000
1063    0.000000
2504    0.250000
124     0.000000
5338    0.000000
4213    0.000000
1924    0.000000
3669    0.000000
4568    0.000000
4393    0.000000
4832    0.000000
2479    0.000000
5736    0.000000
4685    0.000000
3249    0.000000
2116    0.000000
4124    0.000000
1584    0.000000
2582    0.000000
2142    0.000000
3285    0.000000
3578    0.000000
3245    0.000000
99      0.000000
5894    0.000000
4263    0.000000
2249    0.000000
2329    0.000000
334     0.000000
5364    0.000000
4107    0.000000
4992    0.0000

In [268]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
#from sklearn.mixture import GMM

def PCA_call():
    
    pca = PCA(n_components=2)
    projected_data = pca.fit_transform(train_data1)
    plt.figure(figsize=(12,10))
    
    # Create an appropriate color map to be used for positive and negative cases
    colors = {False: 'blue', True: 'red'}
    plt.scatter(projected_data[:,0], projected_data[:,1], color = [colors[x] for x in train_labels])
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.show()
    
PCA_call()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### Logistic Regression Model

Our dataset has now transformed from 17 features to 198 features. Let us fit a Logistic Regression model on this dataset, and evaluate the model against the validation dataset.

In [267]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(C=0.1,penalty='l2',random_state=42)
logistic.fit(train_data1, train_labels)
y_pred = logistic.predict(dev_data1)
evaluate(class_names,dev_labels,y_pred)
print accuracy_score(dev_labels, y_pred)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

We are getting an roc_auc_score of 63%. A GridSearch to find the best value of C has already been incorporated. 

Let us view the feature importances from our Logistic Regression Model.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(train_data1, train_labels)
y_pred = rfc.predict(dev_data1)
evaluate(class_names,dev_labels,y_pred)
print accuracy_score(dev_labels, y_pred)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(train_data1, train_labels)
y_pred = knn.predict(dev_data1)
evaluate(class_names,dev_labels,y_pred)
print accuracy_score(dev_labels, y_pred)

In [ ]:
feature_importances={name:importance for name, importance in sorted(zip(train_data1.columns, logistic.coef_[0]), key=lambda tup: tup[1], reverse=True)}
feature_importances_df=pd.DataFrame.from_dict(feature_importances, orient='index')
feature_importances_df.columns = ['Importance']
pd.options.display.max_rows = 4000
feature_importances_df.round(4).sort_values(by=['Importance'],ascending=False)

We can see that certain features have a strong positive and negative influence in our model. The has_image, requester_number_of_posts_on_raop_at_request, certain days like week 26 which is mostly likely the July 4th weekend in the US and week 52 which is the week between Christmas and the New Year, reciprocity etc. that have a strong positive influence and zero_subreddits, negative_sentiment morning post, day_28 post etc. that have a strong negative influence.

## Building an Ensemble Classifier

## Kaggle Submission

## Conclusion